In [ ]:
import notebook_common
import jaconv
import os
from importlib import reload

reload(notebook_common)
from notebook_common import (
    # ------------------function------------------
    # read_excel,
    to_csv_sig,
    # ------------------2400 word------------------
    # r_word_2400_xlsx,
    # w_word_2400_csv,
    # w_word_2400_with_guide_csv,
    w_word_2400_without_guide_csv,
    # ------------------书面语相关------------------
    # 解析书面语正则
    # pattern_table_written,
    # pattern_td_written,
    # r_written_freq_nlt_with_enter_txt,
    # w_written_freq_nlt_with_enter_csv,
    w_written_freq_nlt_sumed_csv,
    # ------------------口语相关------------------
    # 解析口头语正则
    # pattern_table_spoken,
    # pattern_td_spoken,
    # r_spoken_freq_nlb_with_enter_txt,
    # w_spoken_freq_nlb_with_enter_csv,
    # w_spoken_freq_nlb_with_enter_filtered_csv,
    # w_spoken_freq_nlb_with_enter_filtered_hrt_csv,
    w_spoken_freq_nlb_sumed_csv,
    # ------------------合并书面语和口语词频------------------
    w_merged_freq_sumed_csv,
    w_raw_merged_freq_sumed_csv,
    w_word_raw_merged_freq_sumed_csv,
    # ------------------公共python库------------------
    pd,
    np,
    p,
    l,
)

p("Import common module")

In [ ]:
df_spoken = pd.read_csv(
    w_spoken_freq_nlb_sumed_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    index_col=["headword", "hira"],
)

df_written = pd.read_csv(
    w_written_freq_nlt_sumed_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    index_col=["headword", "hira"],
)


df_w_word_2400_without_guide_csv = pd.read_csv(
    w_word_2400_without_guide_csv,
    encoding="utf-8-sig",
    on_bad_lines="skip",
    engine="python",
    index_col=0,
)
l("df_spoken")
df_spoken

In [ ]:
l("df_written")
df_written

In [ ]:
l("df_w_word_2400_without_guide_csv")
df_w_word_2400_without_guide_csv

In [ ]:
# 输出口语和书面语交集，索引是联合索引 拼写-发音
ids_df_spoken = set(df_spoken.index)
p(f"spoken freq index len:{len(ids_df_spoken)}")
ids_df_written = set(df_written.index)
p(f"written freq index len:{len(ids_df_written)}")
common_ids = ids_df_spoken & ids_df_written
p(f"common index len:{len(common_ids)}")

In [ ]:
# 原始书面语和口语词频合并
df_merged_raw = pd.concat([df_spoken, df_written], axis=1, join="outer")

# 空值补0
df_merged_raw["spoken_freq"] = df_merged_raw["spoken_freq"].fillna(0)
df_merged_raw["written_freq"] = df_merged_raw["written_freq"].fillna(0)

df_merged_raw["cnt_headword_hira"] = df_merged_raw.groupby(
    ["headword", "hira"]
).transform("size")
df_merged_raw["cnt_headword"] = df_merged_raw.groupby(["headword"]).transform("size")


# to_csv_sig(df_merged_raw, w_raw_merged_freq_sumed_csv, True)
p("df_merged_raw--ok")
# df_merged_raw

In [ ]:
# 作标准化处理 ，统计计算每4294,0640的词频, 以书面语词频作为基准
sum_spoken_freq = df_merged_raw["spoken_freq"].sum()
sum_written_freq = df_merged_raw["written_freq"].sum()
p(f"sum_spoken_freq:{sum_spoken_freq}")
p(f"sum_written_freq:{sum_written_freq}")
base_spoken_rate = sum_spoken_freq / sum_written_freq
# base_written_rate = sum_written_freq/10000000
p(f"base_spoken_rate:{base_spoken_rate}")
# p(base_written_rate)

In [ ]:
# 关联mp3发音文件
if os.path.exists("d:/tmp/df_ff_5.csv"):
    df_ff_5 = pd.read_csv(
        "d:/tmp/df_ff_5.csv", encoding="utf-8-sig", on_bad_lines="skip", engine="python"
    )
    l("ok")
else:
    df_ff_4 = pd.read_csv(
        "D:/04.japandict/Forvo/ff-4.csv",
        encoding="utf-8-sig",
        on_bad_lines="skip",
        engine="python",
    )
    df_ff_4.dropna(subset=["headword_voice", "mp3_path"], inplace=True)

    df_ff_4["headword_voice"] = df_ff_4["headword_voice"].apply(jaconv.kata2hira)

    df_ff_5 = (
        df_ff_4.groupby("headword_voice")
        .agg({"mp3_path": lambda x: "|".join(x[x != ""])})  # 字符串拼接  # 求平均值
        .reset_index()
    )
    l("df_ff_5")
    to_csv_sig(df_ff_5, "d:/tmp/df_ff_5.csv")

In [ ]:
df_merged_raw

In [ ]:
# 对词频做标准化处理

# df_merged_raw["raw_spoken_freq"] = df_merged_raw["spoken_freq"]
# df_merged_raw["raw_written_freq"] = df_merged_raw["written_freq"]

# df_merged_raw["spoken_freq"] = df_merged_raw["spoken_freq"] / base_spoken_rate


# df_spoken_written_freq_headword
spoken_weight, written_weight = 1, 1

df_merged_raw["fused_freq"] = round(
    df_merged_raw["spoken_freq"] * spoken_weight / base_spoken_rate
    + df_merged_raw["written_freq"] * written_weight
)


df_merged_raw["freq_for_order"] = round(
    df_merged_raw["spoken_freq"] * spoken_weight
    + df_merged_raw["written_freq"] * base_spoken_rate * written_weight
)


# 计算发音的词频占比
df_merged_raw["freq_sum_headword"] = df_merged_raw.groupby("headword")[
    "fused_freq"
].transform("sum")
df_merged_raw["hira_freq_per"] = round(
    df_merged_raw["fused_freq"] / df_merged_raw["freq_sum_headword"], 2
)


# #过滤融合词频>5的
# df_merged_raw=df_merged_raw[df_merged_raw['fused_freq']>1]
# p(len(df_merged_raw))

# 生成平假名发音id，2位数字，从10开始
df_merged_raw = df_merged_raw.sort_values(
    ["headword", "fused_freq"], ascending=[True, False]
)
df_merged_raw["hira_order"] = df_merged_raw.groupby("headword").cumcount() + 10
df_merged_raw = df_merged_raw[df_merged_raw["hira_order"] < 100]
df_merged_raw["hira_order"] = df_merged_raw["hira_order"].astype(str).str.zfill(2)
df_merged_raw.sort_values(by="freq_for_order", ascending=False, inplace=True)


# to_csv_sig(df_merged_raw,"d:/tmp/hira_order.csv",True)


l("ok")

In [ ]:
df_merged_raw

In [ ]:
df_merged_raw_word = df_merged_raw.reset_index(level=1, drop=True)
df_merged_raw_word

In [ ]:
# 词汇以headword分组，统计
df_merged_raw_word_agg = (
    df_merged_raw_word.groupby(df_merged_raw_word.index)
    .agg(
        {
            "fused_freq": lambda x: sum(x),
            "spoken_freq": lambda x: sum(x),
            "written_freq": lambda x: sum(x),
            "freq_for_order": lambda x: sum(x),
        }
    )  # 字符串拼接  # 求平均值
    .reset_index()
)
l("ok")

In [ ]:
# 输出以word为基础的词频表
df_merged_raw_word_agg.sort_values(by="freq_for_order", ascending=False, inplace=True)
start_index = 100000
df_merged_raw_word_agg.index = range(
    start_index, start_index + len(df_merged_raw_word_agg)
)
to_csv_sig(df_merged_raw_word_agg, w_word_raw_merged_freq_sumed_csv, True)
l("df_merged_raw_word_agg--ok")
df_merged_raw_word_agg

In [ ]:
df_merged_raw

In [ ]:
#临时任务
df2400 = df_w_word_2400_without_guide_csv.set_index(['word','hira'],drop=True)
df2400

In [ ]:
#临时任务 计算2400中拼写和发音在 10万的发音表里有多少存在
df_merged_raw_tmp = df_merged_raw.set_index(['headword','hira'],drop=True)
df_merged_raw_tmp




In [ ]:
#临时任务
idx_df2400=set(df2400.index)
l(len(idx_df2400))

idx_df_merged_raw=set(df_merged_raw.index)
l(len(idx_df_merged_raw))
l(len(idx_df2400 & idx_df_merged_raw))


In [ ]:
# 临时任务
df2400_spell = df_w_word_2400_without_guide_csv.set_index(["word"], drop=True)
df2400_spell

In [ ]:
# 临时任务
df_merged_raw_tmp = df_merged_raw.reset_index(level=1)
df_merged_raw_tmp

In [ ]:
# 临时任务
idx_df2400_spell=set(df2400_spell.index)
l(len(idx_df2400_spell))

idx_df_merged_raw_tmp=set(df_merged_raw_tmp.index)
l(len(idx_df_merged_raw_tmp))
l(len(idx_df2400_spell & idx_df_merged_raw_tmp))

In [ ]:
# df_merged_raw 关联 df_merged_raw_word_agg的ID主键
df_merged_raw_word_agg.reset_index(inplace=True)
df_merged_raw_word_agg = df_merged_raw_word_agg.rename(columns={"index": "idx"})
df_merged_raw_word_agg

In [ ]:
# df_merged_raw 关联 df_merged_raw_word_agg的ID主键
df_merged_raw_word_agg_simple = df_merged_raw_word_agg[
    ["headword", "idx", "fused_freq"]
].copy()

df_merged_raw_word_agg_simple.rename(
    columns={"fused_freq": "word_fused_freq"}, inplace=True
)

# df_merged_raw_word_agg_simple

df_merged_raw2 = df_merged_raw.reset_index()
# df_merged_raw2

df_hira = pd.merge(
    df_merged_raw2, df_merged_raw_word_agg_simple, on="headword", how="left"
)

df_hira["idx"] = df_hira["idx"].astype(str)

df_hira["hira_idx"] = df_hira["idx"] + df_hira["hira_order"]
df_hira.sort_values(
    by=["word_fused_freq", "freq_for_order"], ascending=[False, False], inplace=True
)
to_csv_sig(df_hira, "d:/tmp/result.csv", True)
l("df_hira")

In [ ]:
# 关联发音文件 (headword关联)
# df_merged_raw.reset_index(inplace=True)

df_hira["word_voice"] = df_hira["headword"].map(
    df_ff_5.set_index("headword_voice")["mp3_path"]
)
df_hira["hira_voice"] = df_hira["hira"].map(
    df_ff_5.set_index("headword_voice")["mp3_path"]
)
# df_merged_raw_new = pd.merge(
#     df_hira,
#     df_ff_5,
#     left_on="headword",
#     right_on="headword_voice",
#     how="left",
# )
order = [
    "hira_idx",
    "idx",
    "headword",
    "hira",
    "hira_freq_per",
    "word_fused_freq",
    "fused_freq",
    "freq_for_order",
    "word_voice",
    "hira_voice",
    "spoken_freq",
    "written_freq",
    "roman",
    "cnt_headword",
    "cnt_headword_hira",
]

# df_merged_raw_new = df_merged_raw_new[order]
df_hira = df_hira[order]
df_hira_new = df_hira.sort_values(
    by=["word_fused_freq", "freq_for_order"], ascending=[False, False]
)
to_csv_sig(df_hira_new, w_raw_merged_freq_sumed_csv, True)
p("df_hira--ok")
df_hira_new

In [ ]:
"""
2400词汇跟，合并后的书面语和口语的词频表进行左连接，用new_chinese进行连接

"""


df_spoken_written_freq_headword = (
    df_merged_raw.groupby("headword")[["spoken_freq", "written_freq"]]
    .sum()
    .reset_index()
)

# 对词频做标准化处理
df_spoken_written_freq_headword["spoken_freq"] = (
    df_spoken_written_freq_headword["spoken_freq"] / base_spoken_rate
)


# df_spoken_written_freq_headword
spoken_weight = 0.5
written_weight = 0.5
df_spoken_written_freq_headword["fused_freq"] = round(
    df_spoken_written_freq_headword["spoken_freq"] * spoken_weight
    + df_spoken_written_freq_headword["written_freq"] * written_weight
)


# 先用word进行连接
# t1 = pd.merge(df_w_word_2400_without_guide_csv,df_spoken_written_freq_headword,left_on='word',right_on='headword',how='left')
# t1_renamed = t1.rename(columns={'spoken_freq': 'word_spoken_freq', 'written_freq': 'word_written_freq','headword':'word_headword'})
# 再用chinese进行连接
# t2 = pd.merge(t1_renamed,df_spoken_written_freq_headword,left_on='chinese',right_on='headword',how='left')
# t2_renamed = t2.rename(columns={'spoken_freq': 'chinese_spoken_freq', 'written_freq': 'chinese_written_freq','headword':'chinese_headword'})

t1 = pd.merge(
    df_w_word_2400_without_guide_csv,
    df_spoken_written_freq_headword,
    left_on="new_chinese",
    right_on="headword",
    how="left",
)


to_csv_sig(t1, "d:/tmp/a.csv")
p("ok")

In [ ]:
df_merged = pd.concat([df_spoken, df_written], axis=1, join="outer")


def type_dection(spoken, written):
    if spoken > 0 and written > 0:
        return "both"
    elif np.isnan(spoken):
        return "written"
    elif np.isnan(written):
        return "spoken"
    else:
        return "empty"


df_merged["type"] = df_merged.apply(
    lambda x: type_dection(x["spoken_freq"], x["written_freq"]), axis=1
)

# 查看headword重复的次数
df_merged["headword_repetition"] = df_merged.groupby(["headword"]).transform("size")

# 查看reading重复的次数
df_merged["reading_repetition"] = df_merged.groupby(["reading"]).transform("size")

# 只对headword列进行统计
df_merged["gr_by_word_sum_s_freq"] = df_merged.groupby(["headword"])[
    "spoken_freq"
].transform("sum")
df_merged["gr_by_word_sum_w_freq"] = df_merged.groupby(["headword"])[
    "written_freq"
].transform("sum")

# df_merged.index
to_csv_sig(df_merged, w_merged_freq_sumed_csv, True)
p("w_merged_freq_sumed_csv--ok")

In [ ]:
# word_in_headword  #找出2400价目词汇里有多少个在词频里
word2400_set = set(df_w_word_2400_without_guide_csv["new_chinese"])
p(len(word2400_set))

headword_freq_merge_set = set(df_merged.index.get_level_values(0))
p(len(headword_freq_merge_set))

reading_freq_merge_set = set(df_merged.index.get_level_values(1))
p(len(reading_freq_merge_set))

total_word = 0
effect = 0
word_in_headword = 0
word_in_reading = 0
for word in word2400_set:
    total_word += 1
    if word in headword_freq_merge_set:
        effect += 1
        word_in_headword += 1
        continue
    if word in reading_freq_merge_set:
        p(f"reading word:{word}")
        effect += 1
        word_in_reading += 1
        continue

p(f"effect:{effect}")
p(f"total_word:{total_word}")
p(f"word in headword:{word_in_headword}")
p(f"word in reading:{word_in_reading}")

p(effect / total_word)